In [41]:
import ibm_db
from pandas import DataFrame

In [18]:
import ibm_db_dbi as db

In [19]:
conn = db.connect("DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=sfz85754;PWD=j0-9fgh89tfj30b8;", "", "")

In [44]:
df = pd.read_sql("SELECT * from TELLER", conn)

In [59]:
df

,name,language,service,location,availability
0,Danna Jack,"Chinese,Spanish,Turkish","Mortgage,Savings,Deposit",US,1
1,Amit Miller,"Japanese,Italian,Ukrainian","Mortgage,Savings,Pay",US,0
2,Arthur Loya,"English,Italian,Portuguese","Mortgage,Transfer,Deposit",US,0
3,Michal Hare,"Chinese,Italian,Swedish","Card,Savings,Deposit",US,0
4,Vickie Grimes,"Chinese,Hebrew,Swedish","Mortgage,Invest,Deposit",US,1
...,...,...,...,...,...
995,Nicklaus Ferrari,"Japanese,German,Polish","Card,Transfer,Account",US,0
996,Renata Dewitt,"Korean,Spanish,Turkish","Mortgage,Transfer,Deposit",US,0
997,Hubert Duarte,"Korean,German,Persian","Loan,Invest,Account",US,0
998,Iris Romero,"Russian,Italian,Swedish","Loan,Invest,Account",US,0


In [67]:
# preprocess the database
database = df.set_index('name').T.to_dict('dict')

In [68]:
for key, value in database.items():
    value['language'] = [x.rstrip() for x in value['language'].split(',')]
    value['service'] = [x.rstrip() for x in value['service'].split(',')]

In [100]:
# mock query (preprocessed)
query = {'Smith White': {'language': ['Japanese', 'Hindi', 'Turkish'],
  'service': ['Pay', 'Savings', 'Card'],
  'location': 'US'},
        'Jordan Black': {'language': ['English', 'German', 'Italian'],
  'service': ['Deposit', 'Check', 'Card'],
  'location': 'US'}}

In [90]:
import numpy as np
import pandas as pd
import sys
import math
from collections import Counter

from pandas.io import sql

'''
input: user preference as txt file with features in order: 
language, 
service type, 
location
database: bank assistants as dict of dict of lists/int/string

'''



def sort_ranked_list(prefered_list):
    return(sorted(prefered_list, key = lambda x: x[1], reverse = True)[:3])

def select_bank_assistant(name):
    database[name]['availability'] = 0
    return

def finished_assistant(name):
    database[name]['availability'] = 1

def matching_algorithm(query,threshold):
    # set up a threshold for banking assistants to get into the list
    matched = []
    for name in database:
        if not database[name]['availability']:
            continue
        else:
            counterA = set(query['language'])
            counterB = set(database[name]['language'])
            cos_sim_lang = len(counterA.intersection(counterB))/len(counterA)
            counterA = set(query['service'])
            counterB = set(database[name]['service'])
            cos_sim_ser =len(counterA.intersection(counterB))/len(counterA)
            counterA = set(query['location'])
            counterB = set(database[name]['location'])
            cos_sim_loc =len(counterA.intersection(counterB))/len(counterA)
            score = 0.5 * cos_sim_lang + 0.4 * cos_sim_ser + 0.1 * cos_sim_loc
            if score > threshold:
                bank_assistant = name
                matched.append((bank_assistant,score))
    return matched
    

In [111]:
threshold = 0.5
query_result = dict()
for key,value in query.items(): 
    matched = matching_algorithm(value,threshold)
    ranked = sort_ranked_list(matched)
    teller_info = {key:[(x[0],database[x[0]]) for x in ranked]}
    query_result.update(teller_info)
    #print(ranked)

In [113]:
query_result['Smith White']

[('Charley Hong',
  {'language': ['Japanese', 'Spanish', 'Turkish'],
   'service': ['Loan', 'Savings', 'Pay'],
   'location': 'US',
   'availability': 1}),
 ('Shamara Bradbury',
  {'language': ['Japanese', 'German', 'Turkish'],
   'service': ['Card', 'Transfer', 'Pay'],
   'location': 'US',
   'availability': 1}),
 ('Dominque Keck',
  {'language': ['Korean', 'Italian', 'Turkish'],
   'service': ['Card', 'Savings', 'Pay'],
   'location': 'US',
   'availability': 1})]